In [1]:
import pandas as pd
#import pdfplumber
import os
import numpy as np
import re
from tqdm.notebook import tqdm
from lxml import etree
from collections import Counter
import pdftotext
import zipfile

In [6]:
pdfurl = 'https://wobcovid19.rijksoverheid.nl/publicaties/06f19c847169e6fa18283f6e0c899b9a/besluit-op-wob-verzoek-over-reisadvies-kroatie.pdf' 
name = pdfurl.split('/')[-1]
wobfile = '06f19c847169e6fa18283f6e0c899b9a'
pdftitle = (wobfile + '_' + name).lower()
pdftitlestripped = pdftitle.strip('.pdf')

! curl -s {pdfurl} > {pdftitle} ; pdftk {pdftitle} burst output {pdftitlestripped}-%d.pdf ; rm {pdftitle}
! for file in *.pdf ; do pdftotext "$file" ; done ; rm *.pdf ; rm doc_data.txt

In [8]:
## Converting pdf's to text

import csv

reader = csv.DictReader(open('../../data/MetaDataOverviewReport.csv', encoding='latin-1'))
count = 0

for line in reader:
    pdfurl = line[' PdfUrl']
    filename = pdfurl.split('/')[-1]
    pdftitle = (line['WobFile'] + '_' + filename).lower()
    pdftitlestripped = pdftitle.strip('.pdf')
    txturl = '../data/txt/' + pdftitle.strip('.pdf') + '.txt'
    
    ! curl -s {pdfurl} > {pdftitle} ; pdftk {pdftitle} burst output {pdftitlestripped}-%d.pdf ; rm {pdftitle}
    ! for file in *.pdf ; do pdftotext "$file" ; done ; rm *.pdf ; rm doc_data.txt
        
    count+= 1
    print(count)


1
2
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
3
4
5
6
7
8
   e184824227ff92c0ba8d838ea497a5b6_besluit-op-wob-verzoek-over-optreden-minister-voor-rechtsbescherming-in-uitzending-op1-van-18-mei-2020.pdf
   has set an owner password (which is not required to handle this PDF).
   You did not supply this password. Please respect any copyright.
9
10
11
12
13
14
15
16
17
18
19
20
21
   388491f7f645df30409bec6ef461792b_besluit-op-wob-verzoek-over-aantal-aanwezigen-in-kerkelijke-gebouwen.pdf
   has set an owner password (which is not required to handle this PDF).
   You did not supply this password. Please respect any copyright.
22
   388491f7f645df30409bec6ef461792b_03-inventarislijst.pdf
   has set an owner password (which is not required to handle this PDF).
   You did not supply this password. Please respect any copyright.
23
   388491f7f645df30409bec6ef461792b_02-antwoorden-kamervragen-sgp.pdf
   has set an owner password (w

179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
Syntax Error (96594): Unknown operator 'inf'
Syntax Error (96604): Too few (5) args to 'Tm' operator
Syntax Error (179965): Unknown operator 'inf'
Syntax Error (179975): Too few (5) args to 'Tm' operator
Syntax Error (209679): Unknown operator 'inf'
Syntax Error (209690): Too few (5) args to 'Tm' operator
Syntax Error (201253): Unknown operator 'inf'
Syntax Error (201264): Too few (5) args to 'Tm' operator
Syntax Error (257565): Unknown operator 'inf'
Syntax Error (257576): Too few (5) args to 'Tm' operator
Syntax Error (79221): Unknown operator 'inf'
Syntax Error (79236): Too few (5) args to 'Tm' operator
Syntax Error (234639): Unknown operator 'inf'
Syntax Error (234647): Too few (5) a

# Preprocessing work

In [9]:
ocrdata = pd.read_csv('../data/covid19wob_files_df.tar.xz')

FileNotFoundError: [Errno 2] No such file or directory: '../data/covid19wob_files_df.tar.xz'

In [5]:
def clean_split(text):
    
    
    splitted = re.split('\W+', text)
    cleaned = [word.lower() for word in splitted if word]
     
    return cleaned

def count_words(text):
    
    if not isinstance(text, str):
        return 0

    corpus = clean_split(text)
    
    return len(corpus)

def count_characters(text):
    
    if not isinstance(text, str):
        return 0
    
    corpus = clean_split(text)
    
    return sum([len(word) for word in corpus])

def create_bow(text):
    
    if not isinstance(text, str):
        return {}
    
    cleaned = clean_split(text)
    
    return Counter(cleaned)

In [4]:
ocrdata['nr_words'] = ocrdata['text'].map(lambda x: count_words(x))
ocrdata['nr_chars'] = ocrdata['text'].map(lambda x: count_characters(x))

In [ ]:
bows_ocr = {}

for index, row in ocrdata[:-1].iterrows():
    bows_ocr[str(row['name']) + '-' + str(int(row['page']))] = create_bow(row['text'])

In [6]:

dirpath = '../data/txt//'

bows = {}
for txtfile in os.listdir(dirpath):

    contents = open(dirpath+txtfile).read()
    pages = contents.split('\f')

    for pagenumber, page in enumerate(pages):

        bows[txtfile.strip('.txt') + '-' + str(pagenumber+1)] = create_bow(page)


In [8]:
bows['3fa52482438ed25b69b0be300baaf3c0_documenten-276']

Counter({'8': 1,
         'investeringsagenda': 1,
         'bouw': 1,
         'en': 12,
         'leefomgeving': 1,
         'nazending': 1,
         'bzk': 5,
         'buiten': 2,
         'verzoek': 1,
         '9': 3,
         'noodpakket': 3,
         'caribisch': 2,
         'nederland': 4,
         'stuk': 1,
         'biigevoegd': 1,
         'de': 31,
         'bii': 2,
         'agen': 1,
         'punten': 1,
         '3a': 2,
         '38': 1,
         'worden': 4,
         'nagezonden': 1,
         'aanleiding': 1,
         'dient': 2,
         'mede': 1,
         'namens': 1,
         'andere': 1,
         'departementen': 2,
         'een': 6,
         'integrale': 1,
         'claim': 4,
         'in': 10,
         'voor': 17,
         'bestaat': 2,
         'uit': 6,
         'drie': 1,
         'onderdelen': 3,
         'onderdeel': 4,
         '2': 4,
         '3': 3,
         'betreffen': 1,
         'covid': 1,
         '19': 2,
         'noodmaatregelen': 5,
   

In [55]:
bows_ocr = {key: bows_ocr[key] for key in bows_ocr if key in bows}
bows = {key: bows[key] for key in bows if key in bows_ocr}

# Let's compare!

In [56]:
# Should be an empty set!
print(set(bows_ocr) - set(bows))

print(len(bows_ocr), len(bows))

set()
30881 30881


In [63]:
empty_bags = [bag for bag in bows if len(bows[bag]) == 0]
empty_bags_ocr = [bag for bag in bows_ocr if len(bows_ocr[bag]) == 0]

print(f'Pages with no found words (pdf to text): {len(empty_bags)} ({round(len(empty_bags) / len(bows) * 100, 2)}%)')
print(f'Pages with no found words (ocr): {len(empty_bags_ocr)} ({round(len(empty_bags_ocr) / len(bows_ocr) * 100, 2)}%)')

Pages with no found words (pdf to text): 7550 (24.45%)
Pages with no found words (ocr): 345 (1.12%)


## How many increases/decreases?

In [110]:
# Increases
increases = {}

for page in bows:
    
    original = sum(bows[page].values())
    ocr = sum(bows_ocr[page].values())
    
    increases[page] = ocr - original
    
increases_ocr = pd.Series(increases)

In [93]:
print(f"Decreases: {(increases_ocr < 0).sum()} ({round((increases_ocr < 0).mean() * 100, 2)}%)")
print(f"Increases: {(increases_ocr > 0).sum()} ({round((increases_ocr > 0).mean() * 100, 2)}%)")
print(f"Same amount: {(increases_ocr == 0).sum()} ({round((increases_ocr == 0).mean() * 100, 2)}%)")

print(f"Highest increase: {increases_ocr.max()}. Biggest decrease: {increases_ocr.min()}")
print(f"Mean increase: {increases_ocr.mean()}")

increases_ocr.sort_values()

Decreases: 10559 (34.19%)
Increases: 16442 (53.24%)
Same amount: 3880 (12.56%)
Highest increase: 2021. Biggest decrease: -2310
Mean increase: 57.8344613192578


d8d9c5015c9ceb952052f29e1a27ed1f_inventarislijst-deel-1-1                                                           -2310
3fa52482438ed25b69b0be300baaf3c0_documenten-292                                                                     -1245
d8d9c5015c9ceb952052f29e1a27ed1f_inventarislijst-deel-2-1                                                           -1198
3fa52482438ed25b69b0be300baaf3c0_documenten-276                                                                      -936
3fa52482438ed25b69b0be300baaf3c0_documenten-313                                                                      -930
                                                                                                                     ... 
40f5564f839324b9af20c295dd261007_wob-documenten-79                                                                   1255
40f5564f839324b9af20c295dd261007_wob-documenten-78                                                                   1334
8a676a3415d986008b02572c

In [120]:
# Probleem: pdf to text gaat niet altijd lekker... Verklaart veel van de decreases
# Een decrease is dus eigenlijk vaak juist goed...
print(bows['3fa52482438ed25b69b0be300baaf3c0_documenten-276'].most_common(10))
print(bows_ocr['3fa52482438ed25b69b0be300baaf3c0_documenten-276'].most_common(10))

[('n', 165), ('e', 113), ('i', 96), ('l', 67), ('t', 60), ('m', 47), ('de', 45), ('d', 39), ('a', 35), ('k', 31)]
[('de', 31), ('van', 20), ('voor', 17), ('en', 12), ('in', 10), ('te', 10), ('dit', 9), ('het', 8), ('e', 7), ('een', 6)]


## Subsets

In [112]:
subsets = {}

for page in bows:
    
    subsets[page] = set(bows[page].keys()).issubset(set(bows_ocr[page].keys()))
    
subsets = pd.Series(subsets)

In [113]:
print(f"{subsets.sum()} ({round(subsets.sum() / subsets.count() * 100, 2)}%) docs are a subset of the ocr-ed doc")

10705 (34.67%) docs are a subset of the ocr-ed doc
